In [1]:
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1

In [2]:
!pip install boto3 | tail -n 1

In [3]:
# --- 2. Imports ---
import os
import json
import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model

In [4]:
# --- 3. Set up IBM Credentials ---
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": input("Enter your IBM API Key: ").strip()
}

Enter your IBM API Key: DXF0v8D7305fkKYf1wFf-KOqaBA-WB-xlz6hs2mUESK7


In [5]:
project_id = os.environ.get("PROJECT_ID", input("Enter your project_id (or press Enter if none): "))


Enter your project_id (or press Enter if none): 0189bdeb-204a-4c83-a3e9-b2ce61641d97


In [6]:
# --- 4. Connect to COS & Load Dataset ---
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

In [7]:
bucket = input("Enter your COS bucket name: ").strip()
object_key = input("Enter your dataset file name (e.g., legal_sentiment.csv): ").strip()

Enter your COS bucket name: handson-6-27333
Enter your dataset file name (e.g., legal_sentiment.csv): legal_sentiment_dataset (1).csv


In [14]:
body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "_iter_"):
    def _iter_(self): yield from self
    body._iter_ = types.MethodType(_iter_, body)

In [15]:
data = pd.read_csv(body)
data.head()

,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0


In [16]:
# --- 5. Data Cleaning ---
label_map = {-1: 'negative', 0: 'neutral', 1: 'positive'}
data = data.dropna(subset=['Phrase', 'Sentiment'])
data['Sentiment'] = data['Sentiment'].astype(int)
data['Sentiment'] = data['Sentiment'].map(label_map)
print(data['Sentiment'].value_counts())

Sentiment
positive    184
negative    167
neutral     149
Name: count, dtype: int64


In [17]:
# --- 6. Train-Test Split ---
data_train, data_test = train_test_split(data, test_size=0.3, random_state=42, stratify=data['Sentiment'])

In [18]:
# --- 7. Setup IBM Foundation Model ---
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 42,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 5
}

In [19]:
model = Model(
    model_id=ModelTypes.FLAN_T5_XXL,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
model.get_details()

/usr/local/lib/python3.11/dist-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [20]:
# --- 8. Prepare Few-Shot Examples ---
few_shot_examples = []
for sentiment in data_train['Sentiment'].unique():
    samples = data_train[data_train['Sentiment'] == sentiment].sample(2)
    for _, row in samples.iterrows():
        few_shot_examples.append(f"sentence: {row['Phrase']}\nsentiment: {row['Sentiment']}")

few_shot_context = "\n".join(few_shot_examples)

In [23]:
print(few_shot_context)

sentence: The case was transferred to another jurisdiction.
sentiment: neutral
sentence: The court reviewed the documentation.
sentiment: neutral
sentence: The contract is deemed valid and enforceable.
sentiment: positive
sentence: The court finds in favor of the plaintiff.
sentiment: positive
sentence: The plaintiff's claims are dismissed.
sentiment: negative
sentence: The contract is declared void.
sentiment: negative


In [21]:
instruction = """Determine the sentiment of the following sentence (as 'positive', 'negative', or 'neutral')
.Use the examples below as reference:\n" + few_shot_context + "\n"""

# --- 9. Generate Predictions ---
results = []
for text in data_test['Phrase']:
    prompt = instruction + f"\nsentence: {text}\nsentiment:"
    result = model.generate(prompt)['results'][0]['generated_text']
    results.append(result.strip().lower())

In [30]:
# --- 10. Evaluation ---
y_true = data_test['Sentiment'].values
y_pred = results

In [29]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Classification Report:
              precision    recall  f1-score   support

    negative       0.83      1.00      0.91        50
     neutral       1.00      1.00      1.00        45
    positive       1.00      0.82      0.90        55

    accuracy                           0.93       150
   macro avg       0.94      0.94      0.94       150
weighted avg       0.94      0.93      0.93       150



In [27]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Confusion Matrix:
[[50  0  0]
 [ 0 45  0]
 [10  0 45]]


In [32]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
# --- Sentiment Distribution ---
sentiment_counts = Counter(y_pred)
total = sum(sentiment_counts.values())
summary_distribution = {sentiment: f"{(count/total)*100:.2f}%" for sentiment, count in sentiment_counts.items()}
print("\nSentiment Distribution (%):")
print(json.dumps(summary_distribution, indent=2))


Sentiment Distribution (%):
{
  "neutral": "30.00%",
  "positive": "30.00%",
  "negative": "40.00%"
}


In [33]:
# --- Top Words/Phrases for Negative Sentences ---
negative_texts = data_test['Phrase'][np.array(y_pred) == 'negative']
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2), max_features=10)
X = vectorizer.fit_transform(negative_texts)
top_negative_words = vectorizer.get_feature_names_out()
print("\nTop Words/Phrases in Negative Sentences:")
print(top_negative_words)


Top Words/Phrases in Negative Sentences:
['claims' 'claims dismissed' 'contract declared' 'declared void' 'denied'
 'dismissed' 'motion' 'plaintiff' 'plaintiff claims' 'void']


In [38]:
# --- Summary Statement ---
majority_sentiment = max(sentiment_counts, key=sentiment_counts.get)
summary_statement = (
    f"The overall sentiment of this document is predominantly\n **{majority_sentiment}** "
    f"({summary_distribution[majority_sentiment]} of sentences).\n "
    f"Frequent negative themes include: {', '.join(top_negative_words)}."
)
print("\nSummary Statement:")
print(summary_statement)


Summary Statement:
The overall sentiment of this document is predominantly
 **negative** (40.00% of sentences).
 Frequent negative themes include: claims, claims dismissed, contract declared, declared void, denied, dismissed, motion, plaintiff, plaintiff claims, void.


In [39]:
# --- 11. Output Results to File ---
output_df = data_test.copy()
output_df['Predicted Sentiment'] = y_pred
output_df.to_csv("legal_sentiment_results.csv", index=False)
print("Results saved to legal_sentiment_results.csv")


Results saved to legal_sentiment_results.csv


In [40]:
# --- 12. Interactive Testing ---
while True:
    custom_input = input("\nEnter a legal sentence to analyze sentiment (or type 'exit'): ").strip()
    if custom_input.lower() == 'exit':
        break
    custom_prompt = instruction + f"\nsentence: {custom_input}\nsentiment:"
    response = model.generate(custom_prompt)['results'][0]['generated_text']
    print(f"Predicted Sentiment: {response.strip().lower()}")


Enter a legal sentence to analyze sentiment (or type 'exit'): He is a Criminal.
Predicted Sentiment: negative

Enter a legal sentence to analyze sentiment (or type 'exit'): He is innocent.
Predicted Sentiment: positive

Enter a legal sentence to analyze sentiment (or type 'exit'): He is eating.
Predicted Sentiment: neutral

Enter a legal sentence to analyze sentiment (or type 'exit'): exit
